In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/home/talleyrand/Documents/MLOps/MLOps_CPE393-LabExperimentTrackingMLFlow/homework/mlruns/1', creation_time=1747569219987, experiment_id='1', last_update_time=1747569219987, lifecycle_stage='active', name='mlops_nyc_taxi', tags={}>,
 <Experiment: artifact_location='/home/talleyrand/Documents/MLOps/MLOps_CPE393-LabExperimentTrackingMLFlow/homework/mlruns/0', creation_time=1747569219969, experiment_id='0', last_update_time=1747569219969, lifecycle_stage='active', name='Default', tags={}>]

In [3]:
client.create_experiment(name="my-new-experiment")

'2'

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 100",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [5]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: abb5c729e391406f87f67a917a549965, rmse: 26.8264
run id: e69dd25dfb6e4958956f652442872dcb, rmse: 29.1306
run id: fa4abba162144967ab8ca9f9494936d0, rmse: 30.6969
run id: b3eedf646f8847d6ace7eb0909985627, rmse: 30.9214
run id: 80c80929e9594c808ca9d3b9f184a36e, rmse: 31.2167


In [6]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [8]:
run_id = "abb5c729e391406f87f67a917a549965"  #insert your run id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1747572247352, current_stage='None', description=None, last_updated_timestamp=1747572247352, name='nyc-taxi-regressor', run_id='abb5c729e391406f87f67a917a549965', run_link=None, source='/home/talleyrand/Documents/MLOps/MLOps_CPE393-LabExperimentTrackingMLFlow/homework/mlruns/1/abb5c729e391406f87f67a917a549965/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [9]:
#check model version and stages
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


/tmp/ipykernel_30719/1279244624.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [10]:
# model_version = 4
# new_stage = "Staging"
# # Use client.transition_model_version_stage() function to model to staging.

model_version = 1  # ✅ la version que tu viens de créer
new_stage = "Staging"

client.transition_model_version_stage(
    name="nyc-taxi-regressor",
    version=model_version,
    stage=new_stage
)


/tmp/ipykernel_30719/1607668102.py:8: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1747572247352, current_stage='Staging', description=None, last_updated_timestamp=1747572391683, name='nyc-taxi-regressor', run_id='abb5c729e391406f87f67a917a549965', run_link=None, source='/home/talleyrand/Documents/MLOps/MLOps_CPE393-LabExperimentTrackingMLFlow/homework/mlruns/1/abb5c729e391406f87f67a917a549965/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
Based on the results, update the "Production" model version accordingly.
